In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

spark_conf = SparkConf()
spark_conf.setAll([
    ('spark.master', "local[*]"),
    # ('spark.master', "spark://spark-master:7077"),
    ('spark.app.name', 'bigdata-graphx'),
    ('spark.submit.deployMode', 'client'),
    ("spark.jars", "neo4j-connector-apache-spark_2.12-5.0.0_for_spark_3.jar"),
    ("spark.jars.packages", "graphframes:graphframes:0.8.1-spark3.0-s_2.12"),
])

spark = (
    SparkSession
    .builder
    .config(conf=spark_conf)
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-631d6c35-520a-4d1c-bc09-611c2c15b4ca;1.0
	confs: [default]
	found graphframes#graphframes;0.8.1-spark3.0-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 156ms :: artifacts dl 9ms
	:: modules in use:
	graphframes#graphframes;0.8.1-spark3.0-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------------------------

23/01/08 16:02:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
accounts_df = (
    spark.read
    .format("org.neo4j.spark.DataSource")
    .option("url", "bolt://neo4j:7687")
    .option("query", "MATCH (a:Account) RETURN a.address as id")
    .load()
)

transfers_df = (
    spark.read
    .format("org.neo4j.spark.DataSource")
    .option("url", "bolt://neo4j:7687")
    .option("query", "MATCH (src:Account) -[s:Send]-> (t:Transaction) -[r:Receive]-> (dst:Account) RETURN src.address as src, dst.address as dst, t.txid as txid, r.payload as payload")
    .load()
)

In [4]:
accounts_df.show()
transfers_df.show()

+--------------------+
|                  id|
+--------------------+
|addr1qxeqpdjg4lf0...|
|addr1q9nlkdtwwzu6...|
|addr1q84p0m57t68t...|
|addr1qyqhj3jmd4jy...|
|addr1q9s7g67qfy22...|
|addr1q83ef8afqjp3...|
|addr1q9av5zywsu3f...|
|addr1z8nvjzjeydcn...|
|addr1q97saku7cvu2...|
|addr1qydtjg2erj8c...|
|addr1q928k0uj7h9x...|
|addr1q8t09eutlvz9...|
|addr1q9ywq86tytuk...|
|addr1q8q0ysxyyyq3...|
|addr1q8le02asq699...|
|addr1q8lkcrq7gq9n...|
|addr1qx9wsx4lwvzc...|
|addr1q9eesdst9zf5...|
|addr1qyvs2ls0eyat...|
|addr1qxs0exk8phhd...|
+--------------------+
only showing top 20 rows

+--------------------+--------------------+--------------------+--------------------+
|                 src|                 dst|                txid|             payload|
+--------------------+--------------------+--------------------+--------------------+
|addr1q88d9cxu646p...|addr1q8gztq00uarx...|39704e617bfdfd221...|[NATIVE_TOKEN:972...|
|addr1q8ne6wd4wy65...|addr1qyk7e7xc4vc2...|491efc402a77a1357...|[NATIVE_TOKEN:

In [5]:
from graphframes import *
graph = GraphFrame(accounts_df, transfers_df)

/opt/conda/lib/python3.8/site-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [15]:
# самые популярне адреса(зачастую это либо биржевые адреса или контракты)
from pyspark.sql.functions import col, asc, desc, expr, split, sum, count
graph.degrees.sort(col('degree').desc()).show()


+--------------------+------+
|                  id|degree|
+--------------------+------+
|addr1zxgx3far7qyg...|  1534|
|addr1qyht4ja0zcn4...|   864|
|addr1vy4nmtfc4jff...|   833|
|addr1z8snz7c4974v...|   432|
|addr1wxn9efv2f6w8...|   356|
|addr1qxfmh4pcs334...|   351|
|addr1v8pr9mwnqarw...|   262|
|addr1vy3qpx09uscy...|   262|
|addr1qxkmr0m22xeq...|   256|
|addr1q9cwvremt6n3...|   246|
|addr1w9xc5279jqxq...|   236|
|addr1wxgg25t3tk30...|   222|
|addr1qx9wsx4lwvzc...|   209|
|addr1qyt9d2l8fzgr...|   200|
|addr1q85k2eq506ne...|   170|
|addr1w8dmtwktj7lk...|   162|
|addr1wxugd4ufxmtr...|   162|
|addr1w9v5h80g9nk4...|   162|
|addr1qy88xd8f2ex5...|   160|
|addr1w80k8xe6vckm...|   152|
+--------------------+------+
only showing top 20 rows



In [7]:
from pyspark.sql.types import IntegerType

#найти наибольший полученный баланс нативного токена
transfers_df.show()
def parseNativeTransferAmount(transfer):
    return transfer.withColumn("amount", split(expr("filter(payload, x -> x like 'NATIVE_TOKEN:%')")[0], ':').getItem(1).cast(IntegerType()))

transfers_with_nt_amount = transfers_df.transform(lambda transfer: parseNativeTransferAmount(transfer))
transfers_with_nt_amount.show()
# max send amount
transfers_with_nt_amount.groupBy("src").agg(sum("amount").alias("sum_amount")).sort(col("sum_amount").desc()).show()
# max receipt amount
transfers_with_nt_amount.groupBy("dst").agg(sum("amount").alias("sum_amount")).sort(col("sum_amount").desc()).show()

+--------------------+--------------------+--------------------+--------------------+
|                 src|                 dst|                txid|             payload|
+--------------------+--------------------+--------------------+--------------------+
|addr1q88d9cxu646p...|addr1q8gztq00uarx...|39704e617bfdfd221...|[NATIVE_TOKEN:972...|
|addr1q8ne6wd4wy65...|addr1qyk7e7xc4vc2...|491efc402a77a1357...|[NATIVE_TOKEN:181...|
|addr1qxr6yh7uqnk2...|addr1q93hcde039ys...|bf77749b40ef1ea0a...|[NATIVE_TOKEN:110...|
|addr1qxr6yh7uqnk2...|addr1qxr6yh7uqnk2...|bf77749b40ef1ea0a...|[NATIVE_TOKEN:499...|
|addr1q85r7q5nvp75...|addr1q9uahu7qa67e...|3b125da136a7d7bd3...|[NATIVE_TOKEN:213...|
|addr1q85r7q5nvp75...|addr1qyr3qj7g8n6v...|3b125da136a7d7bd3...|[NATIVE_TOKEN:287...|
|addr1w80ptp0qgmck...|addr1q95u7qe0tlgf...|253507ece0a3288c0...|[undefined:200993...|
|addr1w80ptp0qgmck...|addr1wxj88juwkzmp...|253507ece0a3288c0...|[undefined:291200...|
|addr1w80ptp0qgmck...|addr1q95u7qe0tlgf...|253507ece0a

+--------------------+------------+
|                 src|  sum_amount|
+--------------------+------------+
|addr1vy4nmtfc4jff...|215727317697|
|addr1qxfmh4pcs334...| 81393218782|
|addr1zxgx3far7qyg...| 75623903922|
|addr1q8d6clta0nld...| 68673656898|
|addr1qyht4ja0zcn4...| 52667597654|
|addr1qxn74ffy3trv...| 49444770078|
|addr1q95h99mlelmj...| 34761228650|
|addr1qy88xd8f2ex5...| 31528161728|
|addr1q8ej4u8n07cn...| 26789667975|
|addr1wxn9efv2f6w8...| 26415827277|
|addr1z8snz7c4974v...| 26333798827|
|addr1qx53aw7kg445...| 25405518945|
|addr1q8njx3qrdnwj...| 25082199143|
|addr1qx3e3zsgshs7...| 19885916323|
|addr1qytpg7xglqmk...| 19271242929|
|addr1q9lyrt63rrjy...| 19120995480|
|addr1q8unt3hvltd0...| 18190874050|
|addr1qy7fe8s0k4uf...| 17832179142|
|addr1qyt9d2l8fzgr...| 17784852347|
|addr1qxuf9h49w8s6...| 17435416750|
+--------------------+------------+
only showing top 20 rows

+--------------------+------------+
|                 dst|  sum_amount|
+--------------------+------------+
|a

In [16]:
result = graph.stronglyConnectedComponents(maxIter=10)
result.select("*").orderBy(col("component").asc()).show()

#component size
maxComponents = result.select("*").groupBy("component").agg(count("component").alias("component_size")).orderBy(col("component_size").desc())
maxComponents.show()

+--------------------+---------+
|                  id|component|
+--------------------+---------+
|addr1q80wc369ssxh...|        0|
|addr1q8f7wdrwjype...|        1|
|addr1q9694qnumy8d...|        2|
|addr1qy8n3jt3xlza...|        2|
|addr1q8d27ughqsqr...|        2|
|addr1q80qknl5d0gf...|        2|
|addr1q9gj7sjuakh9...|        2|
|addr1qxz3zrldxkkw...|        2|
|addr1z8nvjzjeydcn...|        2|
|addr1q95h99mlelmj...|        2|
|addr1qy7eqehhxr8u...|        2|
|addr1v9wmh4t2yzkc...|        2|
|addr1qxz6ckqnudze...|        2|
|addr1qyn73eft3k49...|        2|
|addr1qyn57yewp6rp...|        2|
|addr1q85wrt8668zc...|        2|
|addr1q8lkcrq7gq9n...|        2|
|addr1q9qv5t8uy73d...|        2|
|addr1qy67xc94auf8...|        2|
|addr1vxawcsy2dlka...|        2|
+--------------------+---------+
only showing top 20 rows



+------------+--------------+
|   component|component_size|
+------------+--------------+
|           2|           232|
| 25769803782|            57|
|171798691851|             4|
|231928233996|             4|
|266287972355|             4|
|188978561037|             3|
|292057776138|             3|
|146028888079|             3|
| 77309411338|             3|
|678604832770|             3|
| 77309411344|             3|
|180388626437|             2|
|532575944716|             2|
|532575944712|             2|
|549755813897|             2|
|532575944708|             2|
|498216206343|             2|
| 68719476736|             2|
|249108103174|             2|
|738734374928|             2|
+------------+--------------+
only showing top 20 rows



In [18]:
result_lpa = graph.labelPropagation(maxIter=5)
result_lpa.select("id", "label").orderBy(col("label").asc()).show()

maxLabelSize = result_lpa.select("*").groupBy("label").agg(count("label").alias("label_size")).orderBy(col("label_size").desc())
maxLabelSize.show()

+--------------------+-----------+
|                  id|      label|
+--------------------+-----------+
|addr1q8kxr929awyw...|          2|
|addr1q9qla7zdctzn...|          4|
|addr1q9ggxajsu5mq...|          5|
|addr1q9uahu7qa67e...| 8589934592|
|addr1qyr3qj7g8n6v...| 8589934592|
|addr1q8wzvy0aw6gv...| 8589934595|
|addr1w82ezrpnpm28...| 8589934595|
|addr1vx42457eqqpm...| 8589934595|
|addr1q9rdxqz49gnk...| 8589934597|
|addr1q85r7q5nvp75...| 8589934599|
|addr1q9hrlkzmp9xe...| 8589934607|
|addr1q8lswlx5q6g4...| 8589934607|
|addr1qx2xmenrh8v8...|17179869184|
|addr1q9salpdlrl3n...|17179869184|
|DdzFFzCqrhsemKpUM...|17179869184|
|DdzFFzCqrhsim3zq4...|17179869184|
|addr1q9qvp3rucn9n...|17179869189|
|addr1qxg424g78057...|17179869189|
|addr1qyed9ytz9pyc...|25769803779|
|addr1qx6t7jww2ph7...|25769803779|
+--------------------+-----------+
only showing top 20 rows

+-------------+----------+
|        label|label_size|
+-------------+----------+
|1357209665543|       245|
| 773094113293|        74|

In [24]:
# Run PageRank for a fixed number of iterations.
results_pageRank = graph.pageRank(resetProbability=0.15, maxIter=10)

results_pageRank.vertices.select("id", "pagerank").orderBy(col("pagerank").desc()).show()
results_pageRank.edges.select("src", "dst", "weight").orderBy(col("weight").desc()).show()

+--------------------+------------------+
|                  id|          pagerank|
+--------------------+------------------+
|addr1zxgx3far7qyg...| 68.80536991172185|
|addr1q9kpuk727hcj...| 59.19939073691068|
|addr1qyhz45m03yvl...|19.026109286158512|
|addr1qyht4ja0zcn4...|18.204268240043117|
|addr1q8sn8j906u0n...| 18.08591102190165|
|addr1wxn9efv2f6w8...|14.297003995657246|
|addr1q9cwvremt6n3...|14.282671380244086|
|addr1vxrmu3m2cc5k...|13.321056717750047|
|addr1qyw0hccydf0q...|11.878058403654988|
|addr1wy30zf2g4xc7...|11.764126739773703|
|addr1q9je93ppzdf4...|10.118839576196006|
|addr1z8snz7c4974v...| 9.292975753532117|
|addr1q8ww79v2crq0...| 8.570625372361388|
|addr1q94m57ke5y9s...| 7.911270754533622|
|addr1qyjjgw7s7jz3...|  7.67717800680187|
|addr1vy4nmtfc4jff...| 7.535797836086664|
|addr1qxkmr0m22xeq...| 7.290317781075104|
|addr1qxtwk96t635f...| 6.941008078280464|
|addr1qxgmk4l5yd25...| 6.931519313061534|
|addr1q8t02rqffs7p...| 6.485503422028095|
+--------------------+------------

+--------------------+--------------------+------+
|                 src|                 dst|weight|
+--------------------+--------------------+------+
|addr1v95xwcwuapa7...|addr1v8px4syex8e0...|   1.0|
|DdzFFzCqrht5e2dCD...|DdzFFzCqrhsuQV5oh...|   1.0|
|addr1vxnn6psd7uy3...|addr1v8px4syex8e0...|   1.0|
|addr1vxe0uuv3n9cp...|addr1vxrmu3m2cc5k...|   1.0|
|addr1vyh3mylxs05e...|addr1v8px4syex8e0...|   1.0|
|DdzFFzCqrhsjPPdb6...|DdzFFzCqrhsuQV5oh...|   1.0|
|DdzFFzCqrht3gcoPx...|DdzFFzCqrhsuQV5oh...|   1.0|
|DdzFFzCqrhsk81BaJ...|DdzFFzCqrhsuQV5oh...|   1.0|
|addr1qxuvuvltyw9w...|addr1qxuvuvltyw9w...|   1.0|
|DdzFFzCqrhskGtyug...|DdzFFzCqrhsuQV5oh...|   1.0|
|addr1qxnq007ftgq0...|addr1qyajqhkv69q3...|   1.0|
|DdzFFzCqrhsmjFurf...|DdzFFzCqrhsuQV5oh...|   1.0|
|addr1q9lnpd9suply...|DdzFFzCqrhswjSC8W...|   1.0|
|DdzFFzCqrhsmuwoha...|DdzFFzCqrhsuQV5oh...|   1.0|
|addr1qyskeltug3em...|addr1q9suc6m5f26h...|   1.0|
|DdzFFzCqrhsxKQpy2...|DdzFFzCqrhsuQV5oh...|   1.0|
|addr1qx9gyavj86yp...|addr1qy9r